<a href="https://colab.research.google.com/github/buddhazhou/python_study/blob/main/stock_backtest_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from posixpath import join
import pandas as pd
import yfinance as yf
import os
from datetime import date

# variable to store today's date
today = date.today()

#os.environ['http_proxy'] = "http://www-proxy.exu.ericsson.se:8080"
#os.environ['https_proxy'] = "https://www-proxy.exu.ericsson.se:8080"


def backtest(initial_investment=10000, weekly_investment=100,ticker='SPY',starttime='2022-01-01', endtime=today):

    # 获取2023年1月的ticker每日收盘价
    #spy_data = yf.download('515390.SS', start='2022-01-01', end='2022-06-31')
    spy_data = yf.download(ticker, start=starttime, end=endtime)
    spy_data['Date'] = pd.to_datetime(spy_data.index)
    spy_data.set_index('Date', inplace=True)
    # Set options to show all data
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    # Print the DataFrame
    #print(spy_data)

    # Reset options to default (optional)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

    #print(spy_data)
    # 这个值是用于计算LSI买入值
    #print(spy_data)
    #print(spy_data.iloc[0]['Close'])

    # 定义策略参数 initial_investment 只和一次性投入有关
    #initial_investment = 10000
    initial_investment = 10000
    #weekly_investment = 2500
    weekly_investment = 500

    # 提取每周数据（假设每周投资一次）
    #weekly_data = spy_data.resample('W').first()
    weekly_data = spy_data.resample('W').last()
    #print(weekly_data)

    #处理nan值
    #weekly_data = weekly_data.fillna(method='ffill')
    weekly_data = weekly_data.dropna()
    #print(weekly_data)


    """
     weekly_data.fillna(method='ffill')

    This line uses the fillna method on the weekly_data DataFrame.
    fillna is used to fill missing values (NaN) in the DataFrame.
    The argument method='ffill' specifies the method used for filling.
    ffill stands for "forward fill". This means it replaces missing values with the value from the previous valid entry in the same column.
    """
    """
     weekly_data = weekly_data.dropna()

    This line uses the dropna method on the weekly_data DataFrame after filling the missing values.
    dropna permanently removes rows or columns containing missing values (NaN) from the DataFrame.
    By default, it removes rows with any missing value.
    """



    # 初始化变量
    dca_investment = 0
    va_investment = 0
    dca_shares = 0
    va_shares = 0
    lsi_shares = 0
    i = 0
    j = 0
    # Lump Sum Investment

    lsi_shares = initial_investment / spy_data.iloc[0]['Close']
    print(f"\nLSI 股数/买入价 : {lsi_shares}/{spy_data.iloc[0]['Close']}")

    # 模拟DCA策略
    for date, row in weekly_data.iterrows():
        i += 1
    #    print(f"date : {date}")
    #    print(f"The Week {i} Close price : {row['Close']}")
    #    print(f"The week {i}_dca_shares_investment : {weekly_investment / row['Close']}")
        dca_shares += weekly_investment / row['Close']
    #    print(f"The total dca_shares after week {i}: {dca_shares}")
        dca_investment += weekly_investment
    #    print(f"The total dca_investment after week {i}: {dca_investment}")

    # 模拟VA策略
    target_value = 0
    for date, row in weekly_data.iterrows():
        j += 1
    #    print(f"date : {date}")
    #    print(f"The Week {j} Close price : {row['Close']}")
        target_value += weekly_investment
    #    print(f"target_value in week {j}: {target_value}")
        current_value = va_shares * row['Close']
    #    print(f"current_value before week {j}: {current_value}")
        investment_needed = target_value - current_value
    #    print(f"investment_needed in week {j}: {investment_needed}")
        va_shares += investment_needed / row['Close']
        va_investment += investment_needed
    #    print(f"total va_shares after week {j}: {va_shares}")
    #    print(f"total va_investment after week {j}: {va_investment}")
    #print(f"va_investment : {va_investment}")
    # 计算组合价值
    final_price = spy_data.iloc[-1]['Close']
    dca_portfolio_value = dca_shares * final_price
    va_portfolio_value = va_shares * final_price
    lsi_portfolio_value = lsi_shares * final_price
    print(f"\n{ticker} total_dca_investment, number of week : {i}")
    print(f"{ticker} Total_dca_investment : {dca_investment}")
    print(f"{ticker} DCA Portfolio Value: {dca_portfolio_value}")
    print(f"{ticker} DCA Portfolio earning: {dca_portfolio_value-dca_investment}, the earning % is : {(dca_portfolio_value-dca_investment)/dca_investment:.2%} ,the annual interest : is {(dca_portfolio_value-dca_investment)/dca_investment/i*52:.2%}")
    print(f"\n{ticker} total_va_investment, number of week : {j}")
    print(f"{ticker} Total_va_investment : {va_investment}")
    print(f"{ticker} VA Portfolio Value: {va_portfolio_value}")
    print(f"{ticker} VA Portfolio earning: {va_portfolio_value-va_investment}, the earning % is {(va_portfolio_value-va_investment)/va_investment:.2%}, the annual interest : is {(va_portfolio_value-va_investment)/va_investment/j*52:.2%}")
    print(f"\n{ticker} initial_investment is : {initial_investment}")
    print(f"{ticker} LSI Portfolio Value: {lsi_portfolio_value}")
    print(f"{ticker} LSI Portfolio earning: {lsi_portfolio_value-initial_investment}, the earning % is {(lsi_portfolio_value-initial_investment)/initial_investment:.2%}")

    print(f"{ticker} 买入手续费 : {i*1.99}")


backtest(10000, 100,'510210.SS')
#backtest(10000, 100,'VOO')
#backtest(initial_investment=10000, weekly_investment=100,ticker='QQQ')
#backtest(initial_investment=10000, weekly_investment=100,ticker='VTI')


In [ ]:
import yfinance as yf
from datetime import datetime

# CREATE TICKER INSTANCE FOR AMAZON
amzn = yf.Ticker("AMZN")
print(id(amzn),type(amzn),amzn)
# GET TODAYS DATE AND CONVERT IT TO A STRING WITH YYYY-MM-DD FORMAT (YFINANCE EXPECTS THAT FORMAT)
end_date = datetime.now().strftime('%Y-%m-%d')
print(end_date)
amzn_hist = amzn.history(start='2022-01-01',end=end_date)

print(amzn_hist)

135888959012528 <class 'yfinance.ticker.Ticker'> yfinance.Ticker object <AMZN>
2024-07-19
                                 Open        High         Low       Close  \
Date                                                                        
2022-01-03 00:00:00-05:00  167.550003  170.703506  166.160507  170.404495   
2022-01-04 00:00:00-05:00  170.438004  171.399994  166.349503  167.522003   
2022-01-05 00:00:00-05:00  166.882996  167.126495  164.356995  164.356995   
2022-01-06 00:00:00-05:00  163.450500  164.800003  161.936996  163.253998   
2022-01-07 00:00:00-05:00  163.839005  165.243500  162.031006  162.554001   
...                               ...         ...         ...         ...   
2024-07-12 00:00:00-04:00  194.800003  196.470001  193.830002  194.490005   
2024-07-15 00:00:00-04:00  194.559998  196.190002  190.830002  192.720001   
2024-07-16 00:00:00-04:00  195.589996  196.619995  192.240005  193.020004   
2024-07-17 00:00:00-04:00  191.350006  191.580002  185.990005  

In [ ]:
# from posixpath import join
import pandas as pd
import yfinance as yf
import os
from datetime import date
from datetime import datetime

def create_ticker_instance(ticker):
    the_ticker = yf.Ticker(ticker)
    print(the_ticker)
    return the_ticker
    # return yf.Ticker(ticker)
    # variable to store today's date
#print(the_ticker)
end_date = datetime.now().strftime('%Y-%m-%d')
#ticker_hist = the_ticker.history(start='2022-01-01',end=end_date)
#print(ticker_hist)

ticker_name = input("The ticker name is : ")
create_ticker_instance(ticker_name)


# variable to store today's date
today = date.today()

#os.environ['http_proxy'] = "http://www-proxy.exu.ericsson.se:8080"
#os.environ['https_proxy'] = "https://www-proxy.exu.ericsson.se:8080"


def backtest(initial_investment=10000, weekly_investment=100,ticker='SPY',starttime='2022-01-01', endtime=today):

    # 获取2023年1月的ticker每日收盘价
    #spy_data = yf.download('515390.SS', start='2022-01-01', end='2022-06-31')
    spy_data = yf.download(ticker, start=starttime, end=endtime)
    spy_data['Date'] = pd.to_datetime(spy_data.index)
    spy_data.set_index('Date', inplace=True)
    # Set options to show all data
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    # Print the DataFrame
    #print(spy_data)

    # Reset options to default (optional)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

    #print(spy_data)
    # 这个值是用于计算LSI买入值
    #print(spy_data)
    #print(spy_data.iloc[0]['Close'])

    # 定义策略参数 initial_investment 只和一次性投入有关
    #initial_investment = 10000
    initial_investment = 10000
    #weekly_investment = 2500
    weekly_investment = 500

    # 提取每周数据（假设每周投资一次）
    #weekly_data = spy_data.resample('W').first()
    weekly_data = spy_data.resample('W').last()
    #print(weekly_data)

    #处理nan值
    #weekly_data = weekly_data.fillna(method='ffill')
    weekly_data = weekly_data.dropna()
    #print(weekly_data)


    """
     weekly_data.fillna(method='ffill')

    This line uses the fillna method on the weekly_data DataFrame.
    fillna is used to fill missing values (NaN) in the DataFrame.
    The argument method='ffill' specifies the method used for filling.
    ffill stands for "forward fill". This means it replaces missing values with the value from the previous valid entry in the same column.
    """
    """
     weekly_data = weekly_data.dropna()

    This line uses the dropna method on the weekly_data DataFrame after filling the missing values.
    dropna permanently removes rows or columns containing missing values (NaN) from the DataFrame.
    By default, it removes rows with any missing value.
    """



    # 初始化变量
    dca_investment = 0
    va_investment = 0
    dca_shares = 0
    va_shares = 0
    lsi_shares = 0
    i = 0
    j = 0
    # Lump Sum Investment

    lsi_shares = initial_investment / spy_data.iloc[0]['Close']
    print(f"\nLSI 股数/买入价 : {lsi_shares}/{spy_data.iloc[0]['Close']}")

    # 模拟DCA策略
    for date, row in weekly_data.iterrows():
        i += 1
    #    print(f"date : {date}")
    #    print(f"The Week {i} Close price : {row['Close']}")
    #    print(f"The week {i}_dca_shares_investment : {weekly_investment / row['Close']}")
        dca_shares += weekly_investment / row['Close']
    #    print(f"The total dca_shares after week {i}: {dca_shares}")
        dca_investment += weekly_investment
    #    print(f"The total dca_investment after week {i}: {dca_investment}")

    # 模拟VA策略
    target_value = 0
    for date, row in weekly_data.iterrows():
        j += 1
    #    print(f"date : {date}")
    #    print(f"The Week {j} Close price : {row['Close']}")
        target_value += weekly_investment
    #    print(f"target_value in week {j}: {target_value}")
        current_value = va_shares * row['Close']
    #    print(f"current_value before week {j}: {current_value}")
        investment_needed = target_value - current_value
    #    print(f"investment_needed in week {j}: {investment_needed}")
        va_shares += investment_needed / row['Close']
        va_investment += investment_needed
    #    print(f"total va_shares after week {j}: {va_shares}")
    #    print(f"total va_investment after week {j}: {va_investment}")
    #print(f"va_investment : {va_investment}")
    # 计算组合价值
    final_price = spy_data.iloc[-1]['Close']
    dca_portfolio_value = dca_shares * final_price
    va_portfolio_value = va_shares * final_price
    lsi_portfolio_value = lsi_shares * final_price
    print(f"\n{ticker} total_dca_investment, number of week : {i}")
    print(f"{ticker} Total_dca_investment : {dca_investment}")
    print(f"{ticker} DCA Portfolio Value: {dca_portfolio_value}")
    print(f"{ticker} DCA Portfolio earning: {dca_portfolio_value-dca_investment}, the earning % is : {(dca_portfolio_value-dca_investment)/dca_investment:.2%} ,the annual interest : is {(dca_portfolio_value-dca_investment)/dca_investment/i*52:.2%}")
    print(f"\n{ticker} total_va_investment, number of week : {j}")
    print(f"{ticker} Total_va_investment : {va_investment}")
    print(f"{ticker} VA Portfolio Value: {va_portfolio_value}")
    print(f"{ticker} VA Portfolio earning: {va_portfolio_value-va_investment}, the earning % is {(va_portfolio_value-va_investment)/va_investment:.2%}, the annual interest : is {(va_portfolio_value-va_investment)/va_investment/j*52:.2%}")
    print(f"\n{ticker} initial_investment is : {initial_investment}")
    print(f"{ticker} LSI Portfolio Value: {lsi_portfolio_value}")
    print(f"{ticker} LSI Portfolio earning: {lsi_portfolio_value-initial_investment}, the earning % is {(lsi_portfolio_value-initial_investment)/initial_investment:.2%}")

    print(f"{ticker} 买入手续费 : {i*1.99}")


backtest(10000, 100,'510210.SS')
#backtest(10000, 100,'VOO')
#backtest(initial_investment=10000, weekly_investment=100,ticker='QQQ')
#backtest(initial_investment=10000, weekly_investment=100,ticker='VTI')

The ticker name is : voo


[*********************100%%**********************]  1 of 1 completed

yfinance.Ticker object <VOO>

LSI 股数/买入价 : 9803.92175195585/1.0199999809265137

510210.SS total_dca_investment, number of week : 128
510210.SS Total_dca_investment : 64000
510210.SS DCA Portfolio Value: 54771.60137577978
510210.SS DCA Portfolio earning: -9228.398624220223, the earning % is : -14.42% ,the annual interest : is -5.86%

510210.SS total_va_investment, number of week : 128
510210.SS Total_va_investment : 73221.75003692965
510210.SS VA Portfolio Value: 63999.99999999999
510210.SS VA Portfolio earning: -9221.750036929654, the earning % is -12.59%, the annual interest : is -5.12%

510210.SS initial_investment is : 10000
510210.SS LSI Portfolio Value: 7215.686423464128
510210.SS LSI Portfolio earning: -2784.313576535872, the earning % is -27.84%
510210.SS 买入手续费 : 254.72


In [ ]:
# from posixpath import join
import pandas as pd
import yfinance as yf
import os
from datetime import date
from datetime import datetime

def create_ticker_instance(ticker):
    the_ticker = yf.Ticker(ticker)
    print(the_ticker)
    return the_ticker
    # return yf.Ticker(ticker)
    # variable to store today's date
#print(the_ticker)
end_date = datetime.now().strftime('%Y-%m-%d')



ticker_name = input("The ticker name is : ")
current_ticker = create_ticker_instance(ticker_name)
ticker_hist = current_ticker.history(start='2022-01-01',end=end_date)
print(ticker_hist)

The ticker name is : VOO
yfinance.Ticker object <VOO>
                                 Open        High         Low       Close  \
Date                                                                        
2022-01-03 00:00:00-05:00  421.038707  422.327018  418.837055  422.307800   
2022-01-04 00:00:00-05:00  423.586412  424.240206  420.356024  422.134674   
2022-01-05 00:00:00-05:00  421.855841  422.461546  413.924051  414.049042   
2022-01-06 00:00:00-05:00  413.606821  416.135355  411.395516  413.491425   
2022-01-07 00:00:00-05:00  413.616440  414.722078  410.722558  412.058929   
...                               ...         ...         ...         ...   
2024-07-12 00:00:00-04:00  512.489990  518.030029  512.169983  514.549988   
2024-07-15 00:00:00-04:00  516.580017  519.119995  514.340027  516.109985   
2024-07-16 00:00:00-04:00  517.330017  519.400024  516.599976  519.039978   
2024-07-17 00:00:00-04:00  513.559998  515.109985  511.559998  511.790009   
2024-07-18 00:00:00-04

In [ ]:
# from posixpath import join
import pandas as pd
import yfinance as yf
import os
from datetime import date
from datetime import datetime

def create_ticker_instance(ticker):
    get_ticker = yf.Ticker(ticker)
    #print(the_ticker)
    return get_ticker
    # return yf.Ticker(ticker)
    # variable to store today's date
#print(the_ticker)
# end_date = datetime.now().strftime('%Y-%m-%d')
# ticker_hist = the_ticker.history(start='2022-01-01',end=end_date)
#print(ticker_hist)


def print_ticker_hist(ticker_hist):
    # Set options to show all data
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    print(ticker_hist)


    # Reset options to default (optional)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

def backtest(initial_investment=10000, weekly_investment=100,ticker='SPY',starttime='2022-01-01', endtime=today):

    # 定义策略参数 initial_investment 只和一次性投入有关
    #initial_investment = 10000
    initial_investment = 10000
    #weekly_investment = 2500
    weekly_investment = 500

    # 这个值是用于计算LSI买入值, LSI 选取start date的收盘价

    """
    ticker_hist: This is your DataFrame that presumably contains historical stock data, including columns like 'Open', 'High', 'Low', 'Close', 'Volume', etc.
    .iloc[]: This is an attribute used for integer-location based indexing in Pandas DataFrames. It allows you to access rows and columns by their numerical position.
    0: This indicates you want to access the row at index position 0, which is the first row of the DataFrame.
    ['Close']: This part selects the 'Close' column from the row you've selected.
    In essence, the entire expression retrieves the value in the 'Close' column of the first row of your ticker_hist DataFrame, giving you the closing price for the earliest trading day in your data.
    """
    print(ticker_hist.iloc[0]['Close'])
    return 0



ticker_name = input("The ticker name is : ")
the_ticker = create_ticker_instance(ticker_name) # Assign the return value of the function to a variable

# Collect backtest period: start date and end date

start_date = input("<format 2000-01-01> The start date  is : ")
end_date = input("<format 2000-01-01> The end date is :  ")
print_all = input("print all data for this ticker :  y/n : ")


if start_date == "":
    print("No start date input,use default date 2020-01-01")
    start_date = "2024-07-01"


if end_date == "":
    print("No end date input,use today")
    end_date = datetime.now().strftime('%Y-%m-%d')

ticker_hist = the_ticker.history(start=start_date,end=end_date) # Now you can use the_ticker here

# index the data by column 'Date'
"""inplace=True: This argument indicates that the change should be made directly to the existing spy_data DataFrame,
  rather than creating a copy of it. Without inplace=True,
  you would need to assign the result back to spy_data like this: spy_data = spy_data.set_index('Date').
"""

# Reset the index to create a 'Date' column
ticker_hist.reset_index(inplace=True)

# Now you can set 'Date' as index
ticker_hist.set_index('Date', inplace=True)


if print_all == "y" or print_all =="":
    print_ticker_hist(ticker_hist)

#print(ticker_hist.index)

#ticker_hist['Date'] = pd.to_datetime(ticker_hist.index)
#print(ticker_hist)

# 定义策略参数 initial_investment 只和一次性投入有关
#initial_investment = 10000
initial_investment = 10000
#weekly_investment = 2500
weekly_investment = 500

# 这个值是用于计算LSI买入值, LSI 选取start date的收盘价

"""
ticker_hist: This is your DataFrame that presumably contains historical stock data, including columns like 'Open', 'High', 'Low', 'Close', 'Volume', etc.
.iloc[]: This is an attribute used for integer-location based indexing in Pandas DataFrames. It allows you to access rows and columns by their numerical position.
0: This indicates you want to access the row at index position 0, which is the first row of the DataFrame.
['Close']: This part selects the 'Close' column from the row you've selected.
In essence, the entire expression retrieves the value in the 'Close' column of the first row of your ticker_hist DataFrame, giving you the closing price for the earliest trading day in your data.
"""
print(f"\nLSI Buy date: {ticker_hist.index[0]}")
print(f"LIS buy price : {ticker_hist.iloc[0]['Close']}")




The ticker name is : VOO
<format 2000-01-01> The start date  is : 
<format 2000-01-01> The end date is :  
print all data for this ticker :  y/n : y
No start date input,use default date 2020-01-01
No end date input,use today
                                 Open        High         Low       Close  \
Date                                                                        
2024-07-01 00:00:00-04:00  501.489990  501.679993  498.600006  501.279999   
2024-07-02 00:00:00-04:00  499.690002  504.609985  499.609985  504.529999   
2024-07-03 00:00:00-04:00  504.220001  507.140015  504.220001  506.809998   
2024-07-05 00:00:00-04:00  507.130005  510.100006  506.489990  509.839996   
2024-07-08 00:00:00-04:00  510.500000  511.209991  509.320007  510.329987   
2024-07-09 00:00:00-04:00  511.209991  512.010010  510.540009  510.890015   
2024-07-10 00:00:00-04:00  511.980011  516.179993  511.690002  515.809998   
2024-07-11 00:00:00-04:00  516.000000  516.789978  510.829987  511.390015   
2024-